# **1. Perkenalan Dataset**


## Dataset Taxi trip data NYC

## Link Kaggle: https://www.kaggle.com/datasets/anandaramg/taxi-trip-data-nyc

Dataset ini menyajikan analisis komparatif yang mendalam antara layanan taksi konvensional (Yellow Cabs) dan Uber di New York City, menyoroti perbedaan signifikan dalam struktur penetapan harga, aksesibilitas, dan pengalaman pengguna. Informasi di dalamnya mencakup mekanisme perhitungan tarif berdasarkan waktu dan jarak, perbandingan antara surge pricing Uber dengan biaya tambahan jam sibuk taksi, serta regulasi khusus untuk rute bandara seperti JFK dan Newark. Selain aspek ekonomi, dataset ini juga menguraikan variasi jenis kendaraan, metode pembayaran, etika pemberian tip, serta prosedur layanan pelanggan dan penanganan keluhan, memberikan gambaran holistik mengenai opsi transportasi mana yang lebih efisien dan ekonomis bagi penumpang dalam berbagai kondisi lalu lintas.


# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning.

In [11]:
import pandas as pd
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Konfigurasi plot
sns.set(style="whitegrid")
%matplotlib inline

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

In [12]:
# Load dataset
df = pd.read_csv('../taxi_tripdata_raw/taxi_tripdata_raw.csv')
# Tampilkan 5 baris teratas
df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_9192\3447802644.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../taxi_tripdata_raw/taxi_tripdata_raw.csv')


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,1.0,2021-07-01 00:30:52,2021-07-01 00:35:36,N,1.0,74,168,1.0,1.20,6.0,0.5,0.5,0.00,0.0,NaN,0.3,7.30,2.0,1.0,0.0
1,2.0,2021-07-01 00:25:36,2021-07-01 01:01:31,N,1.0,116,265,2.0,13.69,42.0,0.5,0.5,0.00,0.0,NaN,0.3,43.30,2.0,1.0,0.0
2,2.0,2021-07-01 00:05:58,2021-07-01 00:12:00,N,1.0,97,33,1.0,0.95,6.5,0.5,0.5,2.34,0.0,NaN,0.3,10.14,1.0,1.0,0.0
3,2.0,2021-07-01 00:41:40,2021-07-01 00:47:23,N,1.0,74,42,1.0,1.24,6.5,0.5,0.5,0.00,0.0,NaN,0.3,7.80,2.0,1.0,0.0
4,2.0,2021-07-01 00:51:32,2021-07-01 00:58:46,N,1.0,42,244,1.0,1.10,7.0,0.5,0.5,0.00,0.0,NaN,0.3,8.30,2.0,1.0,0.0


# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [13]:
# Cek tipe data dan missing values
df.info()

print("\nJumlah Missing Values per Kolom:")
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83691 entries, 0 to 83690
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorID               51173 non-null  float64
 1   lpep_pickup_datetime   83691 non-null  object 
 2   lpep_dropoff_datetime  83691 non-null  object 
 3   store_and_fwd_flag     51173 non-null  object 
 4   RatecodeID             51173 non-null  float64
 5   PULocationID           83691 non-null  int64  
 6   DOLocationID           83691 non-null  int64  
 7   passenger_count        51173 non-null  float64
 8   trip_distance          83691 non-null  float64
 9   fare_amount            83691 non-null  float64
 10  extra                  83691 non-null  float64
 11  mta_tax                83691 non-null  float64
 12  tip_amount             83691 non-null  float64
 13  tolls_amount           83691 non-null  float64
 14  ehail_fee              0 non-null      float64
 15  im

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning.

Jika Anda menggunakan data teks, data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah. Khususnya ketika kami menggunakan data tidak terstruktur.

In [14]:
initial_shape = df.shape

if 'lpep_pickup_datetime' in df.columns:
    pickup_col = 'lpep_pickup_datetime'
    dropoff_col = 'lpep_dropoff_datetime'
elif 'tpep_pickup_datetime' in df.columns:
    pickup_col = 'tpep_pickup_datetime'
    dropoff_col = 'tpep_dropoff_datetime'
else:
    raise ValueError("Kolom datetime tidak ditemukan! Cek nama header di CSV.")

df[pickup_col] = pd.to_datetime(df[pickup_col])
df[dropoff_col] = pd.to_datetime(df[dropoff_col])

# Hitung durasi perjalanan (menit)
df['trip_duration'] = (df[dropoff_col] - df[pickup_col]).dt.total_seconds() / 60

# Definisi fitur & target
target_col = 'fare_amount'
feature_cols = ['passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'trip_duration']

# Ambil kolom relevan
relevant_cols = feature_cols + [target_col]
missing_cols = [c for c in relevant_cols if c not in df.columns]
if missing_cols:
    print(f"[WARNING] Kolom berikut hilang dari data: {missing_cols}")
    # lanjut hanya dengan kolom yang ada
    relevant_cols = [c for c in relevant_cols if c in df.columns]

df_clean = df[relevant_cols].copy()

# Cleaning rows
df_clean = df_clean.dropna()

# Filter logis dasar
if set(['fare_amount', 'trip_distance', 'trip_duration']).issubset(df_clean.columns):
    df_clean = df_clean[
        (df_clean['fare_amount'] > 0) &
        (df_clean['trip_distance'] > 0) &
        (df_clean['trip_duration'] > 0) &
        (df_clean['trip_duration'] <= 180)  # max 3 jam
    ]

# Filter passenger (optional)
if 'passenger_count' in df_clean.columns:
    df_clean = df_clean[(df_clean['passenger_count'] > 0) & (df_clean['passenger_count'] <= 6)]
    df_clean['passenger_count'] = df_clean['passenger_count'].astype(int)

print(f"[INFO] Cleaning selesai. {initial_shape} -> {df_clean.shape}")

if df_clean.empty:
    raise ValueError("Data kosong setelah cleaning! Cek logika filter Anda.")

print(f"Data Cleaned: {df_clean.shape}")
df_clean.head()

[INFO] Cleaning selesai. (83691, 20) -> (47774, 6)
Data Cleaned: (47774, 6)


,passenger_count,trip_distance,PULocationID,DOLocationID,trip_duration,fare_amount
0,1,1.20,74,168,4.733333,6.0
1,2,13.69,116,265,35.916667,42.0
2,1,0.95,97,33,6.033333,6.5
3,1,1.24,74,42,5.716667,6.5
4,1,1.10,42,244,7.233333,7.0


In [15]:
output_dir = '../taxi_tripdata_preprocessing'
os.makedirs(output_dir, exist_ok=True)

train_df, test_df = train_test_split(df_clean, test_size=0.2, random_state=42)

train_path = os.path.join(output_dir, 'train.csv')
test_path = os.path.join(output_dir, 'test.csv')
train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

print(f"Lokasi Train: {train_path} (Jumlah baris: {len(train_df)})")
print(f"Lokasi Test: {test_path} (Jumlah baris: {len(test_df)})")

Lokasi Train: ../taxi_tripdata_preprocessing\train.csv (Jumlah baris: 38219)
Lokasi Test: ../taxi_tripdata_preprocessing\test.csv (Jumlah baris: 9555)
